In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
path = 'drive/MyDrive/Research/data_collection/final_cleaned/final_dataset.csv'
df = pd.read_csv(path)
df.head()

,fcp,interactive,srt,dom_size,but,fmp,tbt,tbw,fci,cls,lcp,ux_index,fid_category,lcp_category,fcp_category,cls_category
0,810.0,1623.000353,148.574,222,95.844,810.0,1.0,964194,1582.0,0.000000,890.0,810.000000,1,1,2,1
1,791.0,1938.500000,34.172,1049,574.117,1028.5,134.0,2463660,1604.0,0.000000,1623.0,1313.142001,1,1,2,1
2,660.0,1141.000000,309.005,1814,217.650,813.0,68.0,571401,1006.0,0.000800,1440.0,999.346206,1,1,2,1
3,655.0,3584.500000,348.280,1413,1686.482,726.0,484.0,1628682,3204.0,0.028540,820.0,1023.158693,1,2,2,1
4,770.0,3351.000000,775.957,1156,1554.168,770.0,537.0,1737243,3071.0,0.033484,2881.0,1869.895174,1,2,2,2


In [ ]:
# Computing IQR
Q1 = df['ux_index'].quantile(0.25)
Q3 = df['ux_index'].quantile(0.75)
IQR = Q3 - Q1

# Filtering Values between Q1-1.5IQR and Q3+1.5IQR
df = df.query('(@Q1 - 1.5 * @IQR) <= ux_index <= (@Q3 + 1.5 * @IQR)')

In [ ]:
df_ufs = df.drop(columns=['cls', 'dom_size', 'fid_category', 'lcp_category', 'fcp_category', 'cls_category'], axis = 1)
df_ufs.head()

,fcp,interactive,srt,but,fmp,tbt,tbw,fci,lcp,ux_index
0,810.0,1623.000353,148.574,95.844,810.0,1.0,964194,1582.0,890.0,810.000000
1,791.0,1938.500000,34.172,574.117,1028.5,134.0,2463660,1604.0,1623.0,1313.142001
2,660.0,1141.000000,309.005,217.650,813.0,68.0,571401,1006.0,1440.0,999.346206
3,655.0,3584.500000,348.280,1686.482,726.0,484.0,1628682,3204.0,820.0,1023.158693
4,770.0,3351.000000,775.957,1554.168,770.0,537.0,1737243,3071.0,2881.0,1869.895174


# UFS

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# reporductibility
SEED = 1
# instantiate
rf = RandomForestRegressor(random_state = SEED)

In [ ]:
from sklearn.model_selection import train_test_split

X = df_ufs.drop(columns=['ux_index'], axis = 1)
y = df_ufs['ux_index']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=SEED)

In [ ]:
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import GridSearchCV

In [ ]:
params_rf = {
    'n_estimators': [160, 180, 300],
    'max_depth': range(20, 25, 1),
    'min_samples_leaf': range(1, 10, 1),
    'max_features': ['auto', 'log2', 'sqrt'],
}

In [ ]:
grid_rf = GridSearchCV(
  estimator = rf,
  param_grid = params_rf,
  cv = 3,
  scoring = 'neg_mean_squared_error',
  verbose = 1,
  n_jobs = -1
)

In [ ]:
grid_rf.fit(X_train, y_train)

Fitting 3 folds for each of 405 candidates, totalling 1215 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 40.4min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 105.7min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 225.2min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 413.0min
[Parallel(n_jobs=-1)]: Done 1215 out of 1215 | elapsed: 596.2min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=1,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jobs=

In [ ]:
print('hi')

hi


In [ ]:
grid_rf.best_params_

{'max_depth': 23,
 'max_features': 'log2',
 'min_samples_leaf': 1,
 'n_estimators': 300}

In [ ]:
best_model = grid_rf.best_estimator_

y_pred = best_model.predict(X_test)